# Cubs vs White Sox Simulation with Data + History

This notebook loads 2025 batting, pitching, and fielding data for the Cubs and White Sox, processes team strengths, incorporates historical matchup records (from 2020), and simulates the May 18 matchup.

In [2]:
import pandas as pd
import numpy as np

# Load team stats Excel files
cubs_excel = pd.ExcelFile("cubs.xlsx")
sox_excel = pd.ExcelFile("white sox.xlsx")

cubs_batting = cubs_excel.parse("batting")
cubs_pitching = cubs_excel.parse("pitching")
cubs_fielding = cubs_excel.parse("fielding")
sox_batting = sox_excel.parse("batting")
sox_pitching = sox_excel.parse("pitching")
sox_fielding = sox_excel.parse("fielding")

In [3]:
# Process batting
def process_batting(df):
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    cols = ['OPS', 'AVG', 'SLG', 'OBP', 'WAR']
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    return df[cols].mean()

cubs_bat = process_batting(cubs_batting)
sox_bat = process_batting(sox_batting)

# Process pitching
def best_pitcher(df):
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df[['ERA', 'WHIP']] = df[['ERA', 'WHIP']].apply(pd.to_numeric, errors='coerce')
    df = df[(df['ERA'] > 0) & (df['WHIP'] > 0)]
    return df.sort_values(by='ERA').iloc[0][['ERA', 'WHIP']]

cubs_pitch = best_pitcher(cubs_pitching)
sox_pitch = best_pitcher(sox_pitching)

# Process fielding
def average_dwar(df):
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df['DWAR'] = pd.to_numeric(df['DWAR'], errors='coerce')
    return df['DWAR'].mean()

cubs_dwar = average_dwar(cubs_fielding)
sox_dwar = average_dwar(sox_fielding)

In [4]:
# Load and analyze matchup history
matchups = pd.read_excel("cubs vs white sox records.xlsx")
matchups['Date'] = pd.to_datetime(matchups['Year'])
matchups['Winner'] = matchups.apply(lambda r: 'Cubs' if r['Cubs'] > r['White Sox'] else 'White Sox', axis=1)

cubs_win_rate = (matchups['Winner'] == 'Cubs').mean()
score_diff = (matchups['Cubs'] - matchups['White Sox']).mean()

print(f"Historical Cubs Win Rate: {cubs_win_rate:.3f}")
print(f"Avg Score Difference (Cubs - Sox): {score_diff:.3f}")

Historical Cubs Win Rate: 0.556
Avg Score Difference (Cubs - Sox): 0.185


In [5]:
# Strength scoring function with history adjustments
def calculate_strength(ops, war, era, whip, dwar, win_rate, score_diff):
    offense = ops * 100
    defense = era * 20 + whip * 10
    fielding = dwar * 10
    war_adj = war * 5
    history_adj = (win_rate - 0.5) * 10 + score_diff * 5
    return offense - defense + fielding + war_adj + history_adj

cubs_strength = calculate_strength(
    cubs_bat['OPS'], cubs_bat['WAR'], cubs_pitch['ERA'], cubs_pitch['WHIP'], cubs_dwar,
    win_rate=cubs_win_rate, score_diff=score_diff)

sox_strength = calculate_strength(
    sox_bat['OPS'], sox_bat['WAR'], sox_pitch['ERA'], sox_pitch['WHIP'], sox_dwar,
    win_rate=1 - cubs_win_rate, score_diff=-score_diff)

print(f"Cubs Strength: {cubs_strength:.2f}")
print(f"White Sox Strength: {sox_strength:.2f}")

Cubs Strength: 37.78
White Sox Strength: 3.51


In [6]:
# Monte Carlo Simulation
def simulate_game(strength1, strength2, sims=10000):
    wins = sum(np.random.uniform(0, strength1 + strength2) < strength1 for _ in range(sims))
    return wins / sims, 1 - wins / sims

cubs_win, sox_win = simulate_game(cubs_strength, sox_strength)

pd.DataFrame({
    "Team": ["Cubs", "White Sox"],
    "Win Probability": [cubs_win, sox_win]
})

,Team,Win Probability
0,Cubs,0.9165
1,White Sox,0.0835
